### Notes for running this workbook.

1. Expects the source audio, including augmented audio to be located at /local-data/birdsong-recognition/train_audio, with each bird in its own folder
2. The remote object storage is expected to be located at /data, which should contain both 
train_data_decibel_20 and val_data_decibel_20. The notebook will create subdirs as needed


In [1]:
### Can probably cut these down as we are no longer
### running the training in this notebook

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import efficientnet.keras as efn 
import librosa
import librosa.display as display
import os
import warnings
import matplotlib
from tqdm import tqdm
import multiprocessing as mp
from shutil import copyfile
import math


from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
from kapre.augmentation import AdditiveNoise
from kapre.time_frequency import Spectrogram

%matplotlib inline

#!rm -r train_data
#!rm -r val_data
#!rm -r models
#!mkdir models

# suppress warnings
warnings.filterwarnings("ignore")

# NOT USING THE GPU
# SOUND_DIR = "/project/data/birdsong-recognition/train_audio/"
# gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(gpus[0], True)

Using TensorFlow backend.


In [2]:
# function for creating Mel Spectrogram
def createMelSpectrogram(input_path, fileName, output_path, saveOrShow=0, output_path_remote = None, resolution = None):
    
    input_file=os.path.join(output_path, fileName)
    output_file=os.path.join(output_path, fileName[:-4] + ".png")
    print(output_file) 

    if ( not( os.path.exists(output_file)) and not(os.path.isdir(output_file))):
        print(f"{output_file} does not exist, and is not a directory")
        # load sound signal
        signal, sr = librosa.load(os.path.join(input_path, fileName), duration=10)

        # print(signal.shape)
        # print(signal.shape[0] / resolution )
        
        # Calculate the n_hop to get the specified resolution
        if (resolution is None):
            num_hops = 256
            resolution = 224
        else:
            num_hops = int(math.ceil(signal.shape[0] / resolution )) # ceil ensures exact resolution
        
        # create Mel Spectrogram
        print(f"Creating spectrogram {output_file}")
        S = Melspectrogram(n_dft=1024, 
                           n_hop=num_hops,
                           input_shape=(1, signal.shape[0]),
                           padding='same', sr=sr, n_mels=224, fmin=1400, fmax=sr/2,
                           power_melgram=2.0, return_decibel_melgram=True,
                           trainable_fb=False, trainable_kernel=False)(signal.reshape(1, 1, -1)).numpy()

        S = S.reshape(S.shape[1], S.shape[2])

        if saveOrShow == 0:   
            matplotlib.image.imsave(output_file, S)
            if (output_path_remote is not None):
                output_file_remote=os.path.join(output_path_remote, fileName[:-4] + ".png")
                print("Remote Output file:", output_file_remote)
                matplotlib.image.imsave(output_file_remote, S)
                
        else:
            #plt.imshow(S)
            #plt.show()
            display.specshow(S, sr=sr)
            plt.show()

## Create Mel-Spectrograms and split for training and validation

In [3]:
# select top 20 birds for training
BIRDS = os.listdir("/local-data/birdsong-recognition/train_audio/")[0:20]
# BIRDS = ['amered']

In [4]:
print(BIRDS)

['amered', 'yelwar', 'semsan', 'horlar', 'vigswa', 'blugrb1', 'cedwax', 'sposan', 'brthum', 'gockin', 'belkin1', 'olsfly', 'whbnut', 'commer', 'wewpew', 'moudov', 'wilsni1', 'pasfly', 'gryfly', 'annhum']


In [5]:
# set the size of spectragrams here. The spectrograms will be
# square, and with 3 colour channels (RGB): (spsize,spsize,3)
# spsize = 224
# spsize = 299
spsize = 331


train_folder = "/local-data/train_data_decibel_20/"
val_folder = "/local-data/val_data_decibel_20/"

train_folder_remote = "/data/train_data_decibel_20/"
val_folder_remote = "/data/val_data_decibel_20/"


if not os.path.exists(train_folder): os.mkdir(train_folder + str(spsize))
if not os.path.exists(val_folder): os.mkdir(val_folder + str(spsize))

In [6]:
%%time


# create train and val spectrogram
np.random.seed(1234)
def process_bird(bird):
    INPUT_DIR = os.path.join("/local-data/birdsong-recognition/train_audio/", bird)
    TRAIN_DIR = os.path.join(train_folder, str(spsize), bird)
    VAL_DIR = os.path.join(val_folder, str(spsize), bird)
    
    TRAIN_DIR_REMOTE = os.path.join(train_folder_remote, str(spsize), bird)
    VAL_DIR_REMOTE = os.path.join(val_folder_remote, str(spsize), bird)
    
    
    TRAIN_DIR_OLD = os.path.join(train_folder, bird)
    VAL_DIR_OLD = os.path.join(val_folder, bird)
    
    # create bird folders if they don't already exist
    if not(os.path.exists(TRAIN_DIR)):
        os.mkdir(TRAIN_DIR)
    
    if not(os.path.exists(VAL_DIR)): 
        os.mkdir(VAL_DIR)
        
    if not(os.path.exists(TRAIN_DIR_REMOTE)):
        os.mkdir(TRAIN_DIR_REMOTE)
    
    if not(os.path.exists(VAL_DIR_REMOTE)): 
        os.mkdir(VAL_DIR_REMOTE)
        
    # to avoid valdiation and training sets having similar recordings, we will
    # separate them based on the core part of the filename: XCnnnnnn. 
    # We keep track of the cores already sent to each of train and validation sets
    
    # First, build the list of core names currently in the validation and training dirs
    # by stripping off the ending.
    
    trn_names = []
    val_names = []
    
   
    for file in [ f for f in os.listdir(TRAIN_DIR_OLD) if f[0] == "X" ]:
        trn_names.append(file.split(".")[0])
    
    for file in [ f for f in os.listdir(VAL_DIR_OLD) if f[0] == "X" ]:
        val_names.append(file.split(".")[0])
    
    # print("trn_names", trn_names)
    # print("val_names", val_names)
    
    # cross check that nothing appears in both lists
    for core_name in val_names:
        if core_name in trn_names:
            print(f"core name {core_name} appears in both training and validation sets")

    # process file-by-file
            
    # look at each of the the original files (starting with "X")
    # if they are already assigned, the spectragram of the file will
    # be sent to the same data set.
    
    for f in [ i for i in os.listdir(INPUT_DIR) if i[0] == "X"]:
        core_name = f.split(".")[0]
        # this is an original file. Check if it has already been assigned
        if core_name in trn_names:
            dest_dir = TRAIN_DIR
            dest_dir_remote = TRAIN_DIR_REMOTE
            print(f"{core_name} already in training dataset")
        elif core_name in val_names:
            print(f"{core_name} already in validation dataset")
            dest_dir = VAL_DIR
            dest_dir_remote = VAL_DIR_REMOTE
        else:
            # this is an original file we haven't assigned to a dataset
            rand = np.random.randint(0, 10)
            if rand <= 7: 
                dest_dir = TRAIN_DIR
                trn_names.append(core_name)
                print(f"Randomly assigning {core_name} to training dataset")
            else:
                dest_dir = VAL_DIR
                val_names.append(core_name)
                print(f"Randomly assigning {core_name} to validation dataset")
        createMelSpectrogram(INPUT_DIR, f, dest_dir, resolution = spsize )
    
    
    #### FOR TESTING ONLY
    return None

    # Now do the same for the generated files

    for f in [ i for i in os.listdir(INPUT_DIR) if i[0] == "A"]:
        core_name = f.split(".")[1] # discards the leading "A."
        
        # check if it has already been assigned.
        if core_name in trn_names:
            dest_dir = TRAIN_DIR
            dest_dir_remote = TRAIN_DIR_REMOTE
            print(f"{core_name} already in training dataset")
        elif core_name in val_names:
            dest_dir = VAL_DIR
            dest_dir_remote = VAL_DIR_REMOTE
            print(f"{core_name} already in validation dataset")
        else:
            # THIS SHOULD NOT HAPPEN!!!
            # if we get here, then the original XCnnnnnn.png has
            # gone missing, so print a warning and assign randomly
            rand = np.random.randint(0, 10)
            if rand <= 7: 
                dest_dir = TRAIN_DIR
                trn_names.append(core_name)
                print(f"WARNING: Original spectrogram not found, randomly assigning augmented files for {core_name} to training dataset")

            else:
                dest_dir = VAL_DIR
                val_names.append(core_name)
                print(f"WARNING: Original spectrogram not found, randomly assigning augmented files for {core_name} to validation dataset")
        createMelSpectrogram(INPUT_DIR, f, dest_dir, resolution = spsize )
            
            

CPU times: user 13 µs, sys: 17 µs, total: 30 µs
Wall time: 31.5 µs


In [7]:
%%time

def main():

    np.random.seed(1234)
    # comment out the next line and uncomment the following one if running
    # on non-hyperthreaded CPU cores. Assigns one thread per physical core
    # reserving one core for OS, IO, compression etc. Cloud does not have
    # hyperthreading, so adjusted for that.
    
    # threads = int((mp.cpu_count() /2) - 1)
    threads = int(mp.cpu_count() -1)
    
    print(f"Using {threads} threads.")
    # Handle single-core machines
    if (threads < 1): threads = 1

    pool = mp.Pool(threads)
    
    pool.map(process_bird, BIRDS)


        
       

if __name__ == "__main__":
    main()
                   
        

Using 63 threads.
XC139433 already in training dataset
/local-data/train_data_decibel_20/331/amered/XC139433.png
/local-data/train_data_decibel_20/331/amered/XC139433.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/amered/XC139433.png
XC321344 already in training dataset
/local-data/train_data_decibel_20/331/amered/XC321344.0.png
/local-data/train_data_decibel_20/331/amered/XC321344.0.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/amered/XC321344.0.png
XC186306 already in training dataset
/local-data/train_data_decibel_20/331/amered/XC186306.0.png
/local-data/train_data_decibel_20/331/amered/XC186306.0.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/amered/XC186306.0.png
XC478597 already in training dataset
/local-data/train_data_decibel_20/331/amered/XC478597.1.png
/local-data/train_data_decibel_20/331/amered/XC478597.1.png 

/local-data/train_data_decibel_20/331/yelwar/XC121473.png does not exist, and is not a directory
XC187783 already in training dataset
/local-data/train_data_decibel_20/331/amered/XC187783.0.png
/local-data/train_data_decibel_20/331/amered/XC187783.0.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/semsan/XC257528.png
XC406233 already in training dataset
/local-data/train_data_decibel_20/331/semsan/XC406233.0.png
/local-data/train_data_decibel_20/331/semsan/XC406233.0.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/horlar/XC293916.png
Creating spectrogram /local-data/train_data_decibel_20/331/yelwar/XC121473.png
Creating spectrogram /local-data/train_data_decibel_20/331/amered/XC187783.0.png
XC345553 already in training dataset
/local-data/train_data_decibel_20/331/horlar/XC345553.0.png
/local-data/train_data_decibel_20/331/horlar/XC345553.0.png does not exist, and is not a directo

/local-data/train_data_decibel_20/331/yelwar/XC479095.1.png
/local-data/train_data_decibel_20/331/yelwar/XC479095.1.png does not exist, and is not a directory
XC406231 already in training dataset
/local-data/train_data_decibel_20/331/semsan/XC406231.1.png
/local-data/train_data_decibel_20/331/semsan/XC406231.1.png does not exist, and is not a directory
XC195835 already in training dataset
/local-data/train_data_decibel_20/331/amered/XC195835.png
/local-data/train_data_decibel_20/331/amered/XC195835.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/horlar/XC345559.1.png
XC484448 already in training dataset
/local-data/train_data_decibel_20/331/horlar/XC484448.0.png
/local-data/train_data_decibel_20/331/horlar/XC484448.0.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/vigswa/XC128945.png
XC482015 already in training dataset
/local-data/train_data_decibel_20/331/vigswa/XC482015.png
/l

/local-data/val_data_decibel_20/331/vigswa/XC348943.png
/local-data/val_data_decibel_20/331/vigswa/XC348943.png does not exist, and is not a directory
XC490508 already in training dataset
/local-data/train_data_decibel_20/331/blugrb1/XC490508.0.png
/local-data/train_data_decibel_20/331/blugrb1/XC490508.0.png does not exist, and is not a directory
XC491076 already in validation dataset
/local-data/val_data_decibel_20/331/yelwar/XC491076.0.png
/local-data/val_data_decibel_20/331/yelwar/XC491076.0.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/semsan/XC406424.0.png
Creating spectrogram /local-data/train_data_decibel_20/331/amered/XC318331.0.png
XC406420 already in validation dataset
/local-data/val_data_decibel_20/331/semsan/XC406420.0.png
/local-data/val_data_decibel_20/331/semsan/XC406420.0.png does not exist, and is not a directory
XC477187 already in training dataset
/local-data/train_data_decibel_20/331/amered/XC477187.0.png
/loc

/local-data/train_data_decibel_20/331/horlar/XC401720.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/semsan/XC264214.0.png
XC144666 already in training dataset
/local-data/train_data_decibel_20/331/amered/XC144666.1.png
/local-data/train_data_decibel_20/331/amered/XC144666.1.png does not exist, and is not a directory
XC406232 already in training dataset
/local-data/train_data_decibel_20/331/semsan/XC406232.1.png
/local-data/train_data_decibel_20/331/semsan/XC406232.1.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/sposan/XC109329.0.png
XC255373 already in training dataset
/local-data/train_data_decibel_20/331/sposan/XC255373.png
/local-data/train_data_decibel_20/331/sposan/XC255373.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/vigswa/XC297568.1.png
Creating spectrogram /local-data/train_data_decibel_20/331/blugrb1/XC301528.

...

Creating spectrogram /local-data/train_data_decibel_20/331/gockin/XC215704.1.png
XC386587 already in training dataset
/local-data/train_data_decibel_20/331/olsfly/XC386587.1.png
/local-data/train_data_decibel_20/331/olsfly/XC386587.1.png does not exist, and is not a directory
XC124840 already in training dataset
/local-data/train_data_decibel_20/331/gryfly/XC124840.png
/local-data/train_data_decibel_20/331/gryfly/XC124840.png does not exist, and is not a directory
XC144683 already in training dataset
/local-data/train_data_decibel_20/331/gockin/XC144683.1.png
/local-data/train_data_decibel_20/331/gockin/XC144683.1.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/moudov/XC476933.0.png
XC142428 already in training dataset
/local-data/train_data_decibel_20/331/moudov/XC142428.0.png
/local-data/train_data_decibel_20/331/moudov/XC142428.0.png does not exist, and is not a directory
Creating spectrogram /local-data/val_data_decibel_20/331/a

/local-data/train_data_decibel_20/331/pasfly/XC136038.0.png does not exist, and is not a directoryXC467716 already in training dataset

/local-data/train_data_decibel_20/331/olsfly/XC467716.1.pngCreating spectrogram /local-data/val_data_decibel_20/331/moudov/XC153639.png

/local-data/train_data_decibel_20/331/olsfly/XC467716.1.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/gockin/XC287785.1.png
XC31137 already in training dataset
/local-data/train_data_decibel_20/331/moudov/XC31137.png
/local-data/train_data_decibel_20/331/moudov/XC31137.png does not exist, and is not a directory
XC252878 already in training dataset
/local-data/train_data_decibel_20/331/gockin/XC252878.1.png
/local-data/train_data_decibel_20/331/gockin/XC252878.1.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/annhum/XC297440.0.png
Creating spectrogram /local-data/train_data_decibel_20/331/commer/XC234075.1.png


/local-data/train_data_decibel_20/331/gockin/XC305875.1.png
XC488397 already in training dataset/local-data/train_data_decibel_20/331/gockin/XC305875.1.png does not exist, and is not a directory

/local-data/train_data_decibel_20/331/commer/XC488397.0.png
/local-data/train_data_decibel_20/331/commer/XC488397.0.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/annhum/XC473411.png
Creating spectrogram /local-data/train_data_decibel_20/331/wilsni1/XC188823.png
Creating spectrogram /local-data/train_data_decibel_20/331/whbnut/XC121070.png
XC297440 already in training dataset
/local-data/train_data_decibel_20/331/annhum/XC297440.1.png
/local-data/train_data_decibel_20/331/annhum/XC297440.1.png does not exist, and is not a directory
XC337959 already in training dataset
/local-data/train_data_decibel_20/331/wilsni1/XC337959.1.png
/local-data/train_data_decibel_20/331/wilsni1/XC337959.1.png does not exist, and is not a directory
XC350797 alre

Creating spectrogram /local-data/train_data_decibel_20/331/annhum/XC457444.png
XC335697 already in training dataset
/local-data/train_data_decibel_20/331/wilsni1/XC335697.1.png
/local-data/train_data_decibel_20/331/wilsni1/XC335697.1.png does not exist, and is not a directory
XC132237 already in training dataset
/local-data/train_data_decibel_20/331/annhum/XC132237.png
/local-data/train_data_decibel_20/331/annhum/XC132237.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/pasfly/XC348141.png
Creating spectrogram /local-data/val_data_decibel_20/331/commer/XC375582.0.png
XC164565 already in training dataset
/local-data/train_data_decibel_20/331/pasfly/XC164565.png
/local-data/train_data_decibel_20/331/pasfly/XC164565.png does not exist, and is not a directory
XC27233 already in validation dataset
/local-data/val_data_decibel_20/331/commer/XC27233.1.png
/local-data/val_data_decibel_20/331/commer/XC27233.1.png does not exist, and is not a 

/local-data/train_data_decibel_20/331/wilsni1/XC325933.1.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/moudov/XC329214.0.png
Creating spectrogram /local-data/val_data_decibel_20/331/annhum/XC160907.png
XC57970 already in training dataset
/local-data/train_data_decibel_20/331/annhum/XC57970.1.pngXC268206 already in validation dataset

/local-data/train_data_decibel_20/331/annhum/XC57970.1.png does not exist, and is not a directory/local-data/val_data_decibel_20/331/moudov/XC268206.png

/local-data/val_data_decibel_20/331/moudov/XC268206.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/wilsni1/XC325933.1.png
XC45549 already in validation dataset
/local-data/val_data_decibel_20/331/wilsni1/XC45549.png
/local-data/val_data_decibel_20/331/wilsni1/XC45549.png does not exist, and is not a directory
Creating spectrogram /local-data/train_data_decibel_20/331/annhum/XC57970.1.png
Creating